In [30]:
import pandas as pd
import os

# Remove unnecessary features

### 1. Tiki data

In [31]:
file_path = 'Data/tiki_products_áo.csv'
df = pd.read_csv(file_path, sep='\t')
df = df.drop(columns=['Link'])
df.to_csv('Data/tiki_products_áo_modified.csv', index=False)

In [32]:
file_path = 'Data/tiki_products_quần.csv'
df = pd.read_csv(file_path, sep='\t')
df = df.drop(columns=['Link'])
df.to_csv('Data/tiki_products_quần_modified.csv', index=False)

In [33]:
file_path = 'Data/tiki_products_thời trang.csv'
df = pd.read_csv(file_path, sep='\t')
df = df.drop(columns=['Link'])
df.to_csv('Data/tiki_products_thời trang_modified.csv', index=False)

In [34]:
file_path = 'Data/tiki_products.csv'
df = pd.read_csv(file_path, sep='\t')
df = df.drop(columns=['Link'])
df.to_csv('Data/tiki_products_modified.csv', index=False)

In [35]:
file_path = 'Data/tiki_products_váy.csv'
df = pd.read_csv(file_path, sep='\t')
df = df.drop(columns=['Link'])
df.to_csv('Data/tiki_products_váy_modified.csv', index=False)

### 2. Uniqlo data

In [36]:
file_path = 'Data/uniqlo_products.csv' 
df = pd.read_csv(file_path)
df = df.drop(columns=['URL'])
df.to_csv('Data/uniqlo_products_modified.csv', index=False)

## 3. Adidas and Nike data

In [41]:
file_path = 'Data/adidas_nikes_products_snaphost_data.csv'
df = pd.read_csv(file_path)
df = df.drop(columns=['description', 'images', 'scraped_at', 'url'])
df.to_csv('Data/adidas_nikes_products_modified.csv', index=False)